In [1]:
using Revise

In [276]:
using RigidBodyDynamics
using Rotations
using AtlasRobot
using LearningMPC.Models
using StaticArrays
using MeshCat
using MeshCatMechanisms
using Blink
using CoordinateTransformations

In [213]:
vis = Visualizer()
open(vis, Window());

Blink.AtomShell.Window(2, Blink.AtomShell.Electron(Process(`/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/deps/atom/electron /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/src/AtomShell/main.js port 2040`, ProcessRunning), TCPSocket(RawFD(59) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#3))), Blink.Page(2, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(67) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 2, Nullable{Any}(true))))

In [303]:
atlas = AtlasRobot.mechanism()

modifications = [
    ("r_hand_mount", "r_hand", RotZYX(π, -π/2, 0), SVector(0, -0.195, 0)),
    ("l_hand_mount", "l_hand", RotZYX(π, -π/2, 0), SVector(0, -0.195, 0)),
    ("r_foot_sole", "r_foot", RotZYX(0., 0., 0.), SVector(0.0426, -0.0017, -0.07645)),
    ("l_foot_sole", "l_foot", RotZYX(0., 0., 0.), SVector(0.0426, 0.0017, -0.07645)),   
]

for (bodyname, basename, rot, trans) in modifications
    base = findbody(atlas, basename)
    frame = CartesianFrame3D(bodyname)
    inertia = SpatialInertia(frame, SDiagonal(0., 0, 0), SVector(0., 0, 0), 0.0)
    body = RigidBody(inertia)
    joint = Joint("$(basename)_to_$(bodyname)", Fixed{Float64}())
    before_joint = Transform3D(frame_before(joint), default_frame(base), rot, trans)
    attach!(atlas, base, body, joint, joint_pose=before_joint)
end

function init_atlas!(state::MechanismState)
    mechanism = state.mechanism
    zero!(state)
    kneebend = 1.1
    hipbendextra = 0.1
    for sideprefix in ('l', 'r')
        knee = findjoint(mechanism, "$(sideprefix)_leg_kny")
        hippitch = findjoint(mechanism, "$(sideprefix)_leg_hpy")
        anklepitch = findjoint(mechanism, "$(sideprefix)_leg_aky")        
        set_configuration!(state, knee, [kneebend])
        set_configuration!(state, hippitch, [-kneebend / 2 + hipbendextra])
        set_configuration!(state, anklepitch, [-kneebend / 2 - hipbendextra])
    end
    floatingjoint = first(out_joints(root_body(mechanism), mechanism))
    set_configuration!(state, floatingjoint, [1; 0; 0; 0; 0; 0; 0.85])
    state
end


init_atlas! (generic function with 1 method)

In [307]:
π/2

1.5707963267948966

In [322]:
boxatlas = mechanism(BoxAtlas());

In [323]:
delete!(vis)
mvis1 = MechanismVisualizer(atlas, Skeleton(randomize_colors=true), vis[:atlas])
mvis2 = MechanismVisualizer(boxatlas, Skeleton(randomize_colors=true), vis[:boxatlas])
set_configuration!(mvis2, configuration(nominal_state(BoxAtlas())))
set_configuration!(mvis1, configuration(init_atlas!(MechanismState(atlas))))
settransform!(vis[:atlas], Translation(-1, 0, 0.))

matching_bodies =  ["pelvis", "r_foot_sole", "l_foot_sole", "r_hand_mount", "l_hand_mount"]

for body in matching_bodies
    for mvis in [mvis1, mvis2]
        setelement!(mvis, default_frame(findbody(mvis.state.mechanism, body)))
    end
end

In [367]:
function bodymap(state, bodynames)
    mechanism = state.mechanism
    nq = num_positions(state)
    nv = num_velocities(state)
    A = zeros(12 * length(bodynames), nq + nv)
    b = zeros(12 * length(bodynames))
    J_q_to_v = RigidBodyDynamics.configuration_derivative_to_velocity_jacobian(state)
    for i in eachindex(bodynames)
        body = findbody(mechanism, bodynames[i])
        path_to_body = path(mechanism, root_body(mechanism), body)
        J_geometric = geometric_jacobian(state, path_to_body)
        J_analytic = Array(J_geometric) * Array(J_q_to_v)
        rows = (12 * (i - 1)) + (1:12)
        A[rows[1:6], 1:nq] .= J_analytic
        A[rows[7:9], nq + (1:nv)] .= angular(J_geometric)
        A[rows[10:12], nq + (1:nv)] .= linear(J_geometric)
        H = transform_to_root(state, body)
        r = RodriguesVec(rotation(H))
        b[rows[1:3]] .= SVector(r.sx, r.sy, r.sz)
        b[rows[4:6]] .= translation(H)
        T = twist_wrt_world(state, body)
        b[rows[7:9]] .= angular(T)
        b[rows[10:12]] .= linear(T)
    end
    b .= b .- A * Vector(state)
    A, b
end

function _unwrap(b1::Real, b2::Real)
    b1 + (mod(b2 - b1 + π, 2π) - π)
end
    

function _unwrap!(b2::AbstractVector, b1::AbstractVector)
    for i in 1:(length(b2) ÷ 12)
        rows = (12 * (i - 1)) + (1:12)
        for row in vcat(rows[1:3], rows[7:9])
            b2[row] = _unwrap(b1[row], b2[row])
            @assert(b1[row] - π <= b2[row] <= b1[row] + π, (b1[row], b2[row]))
        end
    end
end

_unwrap! (generic function with 1 method)

In [379]:
state1 = init_atlas!(MechanismState(atlas))
state2 = nominal_state(BoxAtlas())
matching_bodies =  ["pelvis", "r_foot_sole", "l_foot_sole", "r_hand_mount", "l_hand_mount"]

5-element Array{String,1}:
 "pelvis"      
 "r_foot_sole" 
 "l_foot_sole" 
 "r_hand_mount"
 "l_hand_mount"

In [380]:
for i in 1:100
    x2_prev = Vector(state2)
    A1, b1 = bodymap(state1, matching_bodies)
    A2, b2 = bodymap(state2, matching_bodies)
    _unwrap!(b1, b2)

    p1 = A1 * Vector(state1) + b1
    x2 = (A2' * A2) \ (A2' * p1 - A2' * b2)
    copy!(state2, x2)
    set_configuration!(mvis2, configuration(state2))
    sleep(0.01)
end

In [381]:
configuration(state2)

11-element RigidBodyDynamics.CustomCollections.SegmentedVector{RigidBodyDynamics.JointID,Float64,Base.OneTo{RigidBodyDynamics.JointID},Array{Float64,1}}:
 -1.53178e-5 
  0.83334    
  0.000650046
  1.5772     
  1.57297    
  0.00222622 
  0.000941339
  0.835802   
  0.836441   
  0.820185   
  0.820328   

In [382]:
velocity(state2)

11-element RigidBodyDynamics.CustomCollections.SegmentedVector{RigidBodyDynamics.JointID,Float64,Base.OneTo{RigidBodyDynamics.JointID},Array{Float64,1}}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [203]:
transform_to_root(state1, findbody(atlas, "pelvis"))

Transform3D from "after_pelvis_to_world" to "world":
rotation: 0.0 rad about [1.0, 0.0, 0.0], translation: [0.0, 0.0, 0.0]

In [204]:
transform_to_root(state2, findbody(boxatlas, "pelvis"))

Transform3D from "after_floating_base_dummy_to_pelvis" to "world":
rotation: 0.10506912757465732 rad about [1.0, -6.12323e-17, 0.0], translation: [-1.56213e-17, -0.255114, -0.183755]

# Next steps

Visualize the matching body frames and make sure their definitions line up